# Test VAE CNN1D

https://stackoverflow.com/questions/43103980/one-dimensional-convolutional-variational-autoencoder-in-keras

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
# Keras uses TensforFlow backend as default
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D,UpSampling1D
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist
import h5py

In [10]:
with h5py.File('SLspectra.hdf5', 'r') as hf:
    data_in = hf['flambda'][:]
    wl_in = hf['wl'][:]

In [11]:
data_in.shape

(550, 10000)

In [18]:
data_in = data_in.reshape(data_in.shape[0],data_in.shape[1],1)

In [19]:
data_in.shape

(550, 10000, 1)

In [12]:
NSIZE = data_in.shape[1]

In [ ]:
# Number of convolutional filters to use
filters = 64
# Convolution kernel size
num_conv = 6
# Set batch size
batch_size = 100
# Decoder output dimensionality
decOutput = 10

latent_dim = 5
intermediate_dim = 256
epsilon_std = 1.0
epochs = 5


In [23]:
x = Input(shape=(NSIZE,1))
# Play around with padding here, not sure what to go with.
conv_1 = Conv1D(1,
                kernel_size=num_conv,
                padding='same', 
                activation='relu')(x)
conv_2 = Conv1D(filters,
                kernel_size=num_conv,
                padding='same', 
                activation='relu',
                strides=1)(conv_1)
flat = Flatten()(conv_2) # Since we are passing flat data anyway, we probably don't need this.
hidden = Dense(intermediate_dim, activation='relu')(flat)
z_mean = Dense(latent_dim)(hidden)
z_log_var = Dense(latent_dim)(hidden)

In [24]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim),
                              mean=0., stddev=epsilon_std)
    return z_mean + K.exp(z_log_var ) * epsilon # the original VAE divides z_log_var with two -- why?

# note that "output_shape" isn't necessary with the TensorFlow backend
# so you could write `Lambda(sampling)([z_mean, z_log_var])`
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

In [28]:
# we instantiate these layers separately so as to reuse them later
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(NSIZE, activation='sigmoid')

h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)
x_decoded_mean = Reshape([NSIZE,1])(x_decoded_mean)


In [29]:
def vae_loss(x, x_decoded_mean):
    xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1) # Double check wtf this is supposed to be
    return xent_loss + kl_loss

In [30]:
vae = Model(x, x_decoded_mean)
vae.compile(optimizer='adam', loss=vae_loss) # 'rmsprop'
vae.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 10000, 1)]   0           []                               
                                                                                                  
 conv1d_11 (Conv1D)             (None, 10000, 1)     7           ['input_9[0][0]']                
                                                                                                  
 conv1d_12 (Conv1D)             (None, 10000, 64)    448         ['conv1d_11[0][0]']              
                                                                                                  
 flatten_2 (Flatten)            (None, 640000)       0           ['conv1d_12[0][0]']              
                                                                                            

In [31]:
x_train = data_in 

x_train = x_train.astype('float32') / 255.

print('x_train.shape:', x_train.shape)



x_train.shape: (550, 10000, 1)


In [32]:
N = 1000
epochs = 2
batch_size = int(N/10)
vae.fit(x_train[0:N,:], x_train[0:N,:],
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size)


Epoch 1/2


TypeError: in user code:

    File "/Users/dagoret/opt/anaconda3/envs/fidle23/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/dagoret/opt/anaconda3/envs/fidle23/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/dagoret/opt/anaconda3/envs/fidle23/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Users/dagoret/opt/anaconda3/envs/fidle23/lib/python3.9/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/dagoret/opt/anaconda3/envs/fidle23/lib/python3.9/site-packages/keras/engine/training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "/Users/dagoret/opt/anaconda3/en